# RA2CE feature : Network Simplification applied to Beira Network

This notebook will guide you through the process of downloading a simplified network tailored to a specific hazard extent, while preserving certain attributes like bridges and tunnels as separate entities in the network

## Network Simplification

Simplifying a network can be done in several ways, depending on the specific needs and goals. When you simplify a network, you essentially merge links from intersection to intersection, making the network easier to manage and analyze. RA2CE allows you to simplify the network while considering certain attributes that you may want to exclude during the process.

For example, if you have bridges and tunnels as assets in your network that you want to keep separate from the rest of the network, RA2CE can handle this through a specific workflow.


# Let's start up the model inputs

First import the necessary Modules

In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import box
import rasterio
from matplotlib.colors import ListedColormap
import random
from shapely.ops import transform
import pyproj

RA2CE module imports

In [ ]:
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection)
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import (OsmNetworkWrapper)
from ra2ce.network.network_simplification.snkit_network_wrapper import (SnkitNetworkWrapper)

In [ ]:
root_dir = Path(".//data//network_simplification_excluding_attributes")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
hazard_folder = root_dir / "static" / "hazard" 
tif_files = hazard_folder.glob("*.tif")

#Find the hazard map
for tif_file in tif_files:
    print(tif_file)

## Now we will load in our hazard map and determine the study area based on the map's extent

In [ ]:
# Function to reproject geometry
def reproject_geometry(geom, src_crs, dst_crs):
    project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
    return transform(project, geom)


# Open the raster file
with rasterio.open(tif_file) as src:
    bbox = src.bounds
    bbox_polygon = box(bbox.left, bbox.bottom, bbox.right, bbox.top)
    src_crs = src.crs
    dst_crs = 'EPSG:4326'

    if src_crs.to_string() != dst_crs:
        # Reproject the bounding box polygon to EPSG:4326
        bbox_polygon = reproject_geometry(bbox_polygon, src_crs, dst_crs)
        print("Hazard Map is in a the CRS:", src_crs)
        print("Reprojected the polygon to EPSG:4326")
        
gdf_polygon = gpd.GeoDataFrame(index=[0], geometry=[bbox_polygon], crs=dst_crs)
centroid = gdf_polygon.geometry.centroid.iloc[0]
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)
folium.GeoJson(gdf_polygon).add_to(m)
m
        

## Set netwok parameters and download the network

In [ ]:
#First we define which roads we will want to download from OSM to create a network with
network_section = NetworkSection(
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[RoadTypeEnum.MOTORWAY,
                RoadTypeEnum.MOTORWAY_LINK,
                RoadTypeEnum.TRUNK, 
                RoadTypeEnum.TRUNK_LINK,
                RoadTypeEnum.PRIMARY, 
                RoadTypeEnum.PRIMARY_LINK,
                RoadTypeEnum.SECONDARY,
                RoadTypeEnum.SECONDARY_LINK,
                RoadTypeEnum.TERTIARY,
                RoadTypeEnum.TERTIARY_LINK,
                RoadTypeEnum.ROAD,
                ],
    save_gpkg=True,
    attributes_to_exclude_in_simplification=['bridge', 'tunnel'],    # add the attributes that needs to be excluded when simplifying the network        
                ) 
   
#pass the specified sections as arguments for configuration
network_config_data = NetworkConfigData(
    root_path= root_dir,
    output_path= root_dir/"output",
    network=network_section,
    static_path=root_dir.joinpath('static'),
    )

#download the network based on the polygon extent and the specified road characteristics
graph, _ = OsmNetworkWrapper.get_network_from_polygon(network_config_data, bbox_polygon)

## Read the simplified graph network

We convert the simplified graph (from NetworkX to gpkg) to geopandas.

In [6]:
snkit_network_wrapper = SnkitNetworkWrapper.from_networkx(
            graph,
            column_names_dict=dict(
                node_id_column_name="id",
                edge_from_id_column_name="from_id",
                edge_to_id_column_name="to_id",
            ),
        )

simplified_gdf = snkit_network_wrapper.snkit_network.edges

## Lets visualize the downloaded and simplified network

The attributes we excluded in this case bridges/tunnels are visualized in black. You can observe that the network is segmented from intersection to intersection as well as assets we excluded 

In [ ]:
# Function to generate random colors, excluding white and light gray
def generate_vivid_colors(n):
    colors = []
    for _ in range(n):
        while True:
            r, g, b = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
            # Exclude colors that are too close to white or light gray
            if (r + g + b) < 600:  # Total brightness threshold (255 * 3 = 765 for white)
                colors.append((r, g, b))
                break
    return colors

center = [
    gdf_polygon['geometry'].centroid.y.mean(),
    gdf_polygon['geometry'].centroid.x.mean()
]
map = folium.Map(location=center, zoom_start=12, control_scale=True, tiles="cartodbpositron")

# Create a FeatureGroup for the bridge layer
bridge_layer = folium.FeatureGroup(name="Bridges").add_to(m)
folium.GeoJson(
    simplified_gdf,
    style_function=lambda x: {
        'color': 'black' if x['properties']['bridge'] == 'yes' else 'transparent',
        'fillOpacity': 0.6 if x['properties']['bridge'] == 'yes' else 0.0,
        'weight': 15 if x['properties']['bridge'] == 'yes' else 0
    },
    tooltip=folium.GeoJsonTooltip(fields=['bridge'], aliases=['Bridge Status'])
).add_to(bridge_layer)
bridge_layer.add_to(map)

## Create a FeatureGroup for the simplified network

# Ensure 'rfid' is treated as categorical and create random distinct RGB colors
simplified_gdf['rfid'] = simplified_gdf['rfid'].astype('category')
unique_rfid = simplified_gdf['rfid'].cat.categories
random_colors = generate_vivid_colors(len(unique_rfid))
random_colors_normalized = [(r/255, g/255, b/255) for r, g, b in random_colors]

# Create a ListedColormap with the random RGB colors
cmap = ListedColormap(random_colors_normalized)

network_group = folium.FeatureGroup(name="Simplified network").add_to(m)
simplified_gdf.explore(m=network_group, column='rfid', cmap=cmap, tiles="CartoDB positron", categorical=True, legend=False)
network_group.add_to(map)

# Add Layer Control
folium.LayerControl().add_to(map)

# Show the map
map